In [1]:
%matplotlib inline

In [2]:
import torch 
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [3]:
#To determine if your system supports CUDA
print("==> Check devices..")
if torch.cuda.is_available():
    device = 'cuda'
else:    
    device = 'cpu'
print("Current device: ",device)
if device == 'cuda':
    print("Our selected device: ", torch.cuda.current_device())
    print(torch.cuda.device_count(), " GPUs is available")

==> Check devices..
Current device:  cuda
Our selected device:  0
1  GPUs is available


In [4]:
def get_mean_and_std(dataset):
    '''Compute the mean and std value of dataset.'''
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=1, shuffle=False, num_workers=2)

    mean = torch.zeros(3)
    std = torch.zeros(3)
    print('==> Computing mean and std..')
    for inputs, targets in dataloader:
        for i in range(3):
            mean[i] += inputs[:, i, :, :].mean()
            std[i] += inputs[:, i, :, :].std()
    mean.div_(len(dataset))
    std.div_(len(dataset))
    return mean, std

In [5]:
calculate_mean_std = True

if calculate_mean_std == True:
    #we will calculate mean and std
    #The transform function for train data
    transform_train = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])

    transform_val = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])

    #The transform function for test data
    transform_test = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])

    trainset = torchvision.datasets.ImageFolder(root='../data/cinic10/train', transform=transform_train)
    valset = torchvision.datasets.ImageFolder(root='../data/cinic10/valid', transform=transform_val)
    testset = torchvision.datasets.ImageFolder(root='../data/cinic10/test', transform=transform_test)

    train_mean, train_std = get_mean_and_std(trainset)
    print(train_mean, train_std)
    val_mean, val_std = get_mean_and_std(valset)
    print(val_mean, val_std)
    test_mean, test_std = get_mean_and_std(testset)
    print(test_mean, test_std)
else:
    train_mean, train_std = ([0.4770, 0.4667, 0.4244]), ([0.1884, 0.1852, 0.1874])
    val_mean, val_std = ([0.4769, 0.4663, 0.4240]), ([0.1885, 0.1850, 0.1872])
    test_mean, test_std = ([0.4769, 0.4668, 0.4245]), ([0.1886, 0.1851, 0.1874])

In [6]:
#The transform function for train data
transform_train = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(train_mean, train_std)
])

#The transform function for validation data
transform_val = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(val_mean, val_std)
])

#The transform function for test data
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(test_mean, test_std)
])

In [7]:
trainset = torchvision.datasets.ImageFolder(root='../data/cinic10/train', transform=transform_train)
valset = torchvision.datasets.ImageFolder(root='../data/cinic10/valid', transform=transform_val)
testset = torchvision.datasets.ImageFolder(root='../data/cinic10/test', transform=transform_test)

In [8]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
    shuffle=True, num_workers=2)

valloader = torch.utils.data.DataLoader(valset, batch_size=32,
    shuffle=True, num_workers=2)

testloader = torch.utils.data.DataLoader(testset, batch_size=32,
    shuffle=False, num_workers=2)

In [9]:
classes = ('airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

In [10]:
import torch.nn as nn

In [11]:
model = torchvision.models.resnet34(pretrained=False)
if device == 'cuda':
    model = model.cuda(0)
else:
    model = model.cpu()

In [12]:
import torch.optim as optim

#loss function
criterion = nn.CrossEntropyLoss()
#optimization algorithm
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.001)

In [13]:
for phase in ['train', 'val']:
    if phase == 'train':
        print(phase)    
    else:
        print(phase)

train
val


In [14]:
import copy

best_model = model
best_acc = 0.0
val_acc = 0.0
print(device)

model.train()
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.1, verbose=True)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size = 1, gamma=0.5)

for epoch in range(1):  # loop over the dataset multiple times
        
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train(True)  # Set model to training mode
            dset_loaders = trainloader
        else:
            model.train(False)  # Set model to evaluate mode
            dset_loaders = valloader
            
        running_loss = 0.0
        correct = 0
        
        for i, data in enumerate(dset_loaders, 0):
        
            (inputs, labels) = data
            
            #change the type into cuda tensor 
            if device == 'cuda':
                inputs = inputs.cuda(0)
                labels = labels.cuda(0)
            else:
                inputs = inputs.cpu()
                labels = labels.cpu()

            #print(labels)
            #print(inputs)
        
            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = model(inputs)
            # select the class with highest probability
            _, pred = outputs.max(1)
            # if the model predicts the same results as the true
            # label, then the correct counter will plus 1
            correct += pred.eq(labels).sum().item()
        
            loss = criterion(outputs, labels)
        
            if phase == 'train':
                loss.backward()
                optimizer.step()

            # print statistics
            running_loss += loss.item()
            if (i % 200) == 199:    # print every 200 mini-batches
                print('[%d, %5d]' % (epoch + 1, i + 1))
            
        if phase == 'train':
            epoch_loss = running_loss / len(trainset)
            epoch_acc = correct / len(trainset)
        else:
            epoch_loss = running_loss / len(valset)
            epoch_acc = correct / len(valset)
            val_acc = epoch_acc

        print('{:d} Epoch {} Loss: {:.4f} Acc: {:.4f}'.format(epoch, phase, epoch_loss, epoch_acc))

        # deep copy the model
        if phase == 'val' and epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model = copy.deepcopy(model)
    lr_scheduler.step(val_acc)
print('Finished Training')

model = best_model

cuda
[1,   200]
[1,   400]
[1,   600]
[1,   800]
[1,  1000]
[1,  1200]
[1,  1400]
[1,  1600]
[1,  1800]
[1,  2000]
[1,  2200]
[1,  2400]
[1,  2600]
[1,  2800]
0 Epoch train Loss: 0.0527 Acc: 0.3775
[1,   200]
[1,   400]
[1,   600]
[1,   800]
[1,  1000]
[1,  1200]
[1,  1400]
[1,  1600]
[1,  1800]
[1,  2000]
[1,  2200]
[1,  2400]
[1,  2600]
[1,  2800]
0 Epoch val Loss: 0.0445 Acc: 0.4821
Finished Training


In [17]:
print('==> Saving model..')

#only save model parameters
torch.save(model.state_dict(), './checkpoint.t7')
#you also can store some log information
state = {
    'net': model.state_dict(),
    'acc': 100.*correct/len(trainset),
    'epoch': epoch
}
torch.save(state, './checkpoint.t7')

#save entire model
torch.save(model, './model.pt')

print('Finished Saving')

==> Saving model..
Finished Saving


In [18]:
print('==> Testing model..')
model.eval()

==> Testing model..


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Co

In [21]:
correct = 0
total = 0
running_loss = 0.0
# device == 'cpu'
class_correct = list(0. for i in range(11))
class_total = list(0. for i in range(11))

with torch.no_grad():
    for mini_batch, data in enumerate(testloader, 0):
        images, labels = data

        if device == 'cuda':
            images = images.cuda(0)
            labels = labels.cuda(0)
        else:
            images = images.cpu()
            labels = labels.cpu()
        
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        loss = criterion(outputs, labels)
        running_loss += loss.item()

        c = (predicted == labels).squeeze()
        #print(predicted)
        #print(labels.size(0))
        #print(c)
        for i in range(labels.size(0)):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1
        if(mini_batch % 200 == 199):
            print('[%5d]' % (mini_batch + 1))
            
print('Accuracy of the network on the %d test images: %.2f%%, and loss is: %.3f'
      % (total, 100 * correct / total, running_loss / total))

for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

[  200]
[  400]
[  600]
[  800]
[ 1000]
[ 1200]
[ 1400]
[ 1600]
[ 1800]
[ 2000]
[ 2200]
[ 2400]
[ 2600]
[ 2800]
Accuracy of the network on the 90000 test images: 48.19%, and loss is: 0.045
Accuracy of airplane : 62 %
Accuracy of automobile : 67 %
Accuracy of  bird : 27 %
Accuracy of   cat : 33 %
Accuracy of  deer : 32 %
Accuracy of   dog : 26 %
Accuracy of  frog : 63 %
Accuracy of horse : 70 %
Accuracy of  ship : 46 %
Accuracy of truck : 52 %
